In [1]:
import pandas as pd
import numpy as np

In [5]:
#Загрузчик с диска гугл
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

df_train = pd.read_csv("../content/drive/MyDrive/Colab Notebooks/NLP/7/data/train.csv")
df_test = pd.read_csv("../content/drive/MyDrive/Colab Notebooks/NLP/7/data/test.csv")
df_val = pd.read_csv("../content/drive/MyDrive/Colab Notebooks/NLP/7/data/val.csv")



In [ ]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [8]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['text'] = df_train['text'].apply(preprocess_text)
df_val['text'] = df_val['text'].apply(preprocess_text)
df_test['text'] = df_test['text'].apply(preprocess_text)

In [10]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
#from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [11]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import Constant

In [12]:
text_corpus_train = df_train['text'].values
text_corpus_valid = df_val['text'].values
text_corpus_test = df_test['text'].values

In [13]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_valid)
sequences_test = tokenizer.texts_to_sequences(text_corpus_test)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

X_train = pad_sequences(sequences_train, maxlen=training_length)
X_valid = pad_sequences(sequences_val, maxlen=training_length)



In [14]:
y_train = df_train['class'].values
y_val = df_val['class'].values

In [15]:
model = Sequential()

model.add(
    Embedding(input_dim=word_count,
              input_length=training_length,
              output_dim=30,
              trainable=True,
              mask_zero=True))
model.add(Masking(mask_value=0.0))

model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=512,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/10
319/319 [==============================] - 50s 151ms/step - loss: 0.5612 - accuracy: 0.6976 - val_loss: 0.4930 - val_accuracy: 0.7584
Epoch 2/10
319/319 [==============================] - 51s 158ms/step - loss: 0.2977 - accuracy: 0.8785 - val_loss: 0.5624 - val_accuracy: 0.7437


In [17]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 20ms/step - loss: 0.5883 - accuracy: 0.7331


Test score: 0.5882564187049866
Test accuracy: 0.7330600023269653


RNN

In [18]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
         )
model.add(Masking(mask_value=0.0))
model.add(SimpleRNN(64))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )

In [19]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, 
                    verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 52s 157ms/step - loss: 0.5589 - accuracy: 0.7014 - val_loss: 0.4899 - val_accuracy: 0.7569
Epoch 2/10
319/319 [==============================] - 48s 151ms/step - loss: 0.2863 - accuracy: 0.8827 - val_loss: 0.5555 - val_accuracy: 0.7420


In [20]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 1s 21ms/step - loss: 0.5949 - accuracy: 0.7270


Test score: 0.5948747992515564
Test accuracy: 0.7269761562347412


CNN + RNN

In [27]:
model = Sequential()
model.add(Embedding(input_dim=word_count, 
                    input_length=training_length, 
                    output_dim=30, 
                    trainable=True, 
                    mask_zero=True)
          )
model.add(Masking(mask_value=0.0))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(SimpleRNN(128))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy']
              )

In [28]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(X_train, 
                    y_train, 
                    batch_size=512, 
                    epochs=10, verbose=1, 
                    validation_split=0.1, 
                    callbacks=[early_stopping]
                    )

Epoch 1/10
319/319 [==============================] - 73s 223ms/step - loss: 0.5489 - accuracy: 0.7071 - val_loss: 0.4887 - val_accuracy: 0.7598
Epoch 2/10
319/319 [==============================] - 73s 230ms/step - loss: 0.2811 - accuracy: 0.8839 - val_loss: 0.5521 - val_accuracy: 0.7445


In [29]:
score = model.evaluate(X_valid, y_val, batch_size=512, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

45/45 [==============================] - 2s 42ms/step - loss: 0.6011 - accuracy: 0.7318


Test score: 0.6010779738426208
Test accuracy: 0.731825590133667
